In [1]:
# @title
### EP2 MAC0417 / MAC5768
##################################################################
# AO PREENCHER ESSE CABEÇALHO COM O MEU NOME E O MEU NÚMERO USP,#
# DECLARO QUE SOU O ÚNICO AUTOR E RESPONSÁVEL PELA RESOLUÇÃO #
# DESTE EP. #
# TODAS AS PARTES FORAM DESENVOLVIDAS E IMPLEMENTADAS POR MIM, #
# SEGUINDO AS INSTRUÇÕES E QUE PORTANTO, NÃO CONSTITUEM #
# DESONESTIDADE ACADÊMICA OU PLÁGIO. #
# #
# DECLARO TAMBÉM, QUE SOU RESPONSÁVEL POR TODAS AS CÓPIAS #
# DESSE PROGRAMA, E QUE EU NÃO DISTRIBUI OU FACILITEI A #
# SUA DISTRIBUIÇÃO. ESTOU CIENTE QUE OS CASOS DE PLÁGIO E #
# DESONESTIDADE ACADÊMICA SERÃO TRATADOS SEGUNDO OS CRITÉRIOS #
# DEFINIDOS NO CÓDIGO DE ÉTICA DA USP. #
# #
# ENTENDO QUE JUPYTER NOTEBOOKS SEM ASSINATURA NÃO SERÃO #
# CORRIGIDOS E, AINDA ASSIM, PODERÃO SER PUNIDOS POR #
# DESONESTIDADE ACADÊMICA. #
# #
# #
# Nome : William Garib Gugel
# NUSP : 11238131
# Turma: #
# Prof.: #
##################################################################

In [2]:
!pip install opencv-python
from ImageBase import ImageBase
import numpy as np
from cv2 import imwrite, cvtColor, COLOR_BGR2GRAY, resize

# Amostra da Oganização dos Arquivos

In [3]:
x=ImageBase("./ImageBase/")
x.data.head()

,path,classe,iluminacao,fundo,C,F,H,M,N,O,P,R,T,X,size
id,,,,,,,,,,,,,,,
PCT$E@A2.JPG,ImageBase\Ext\Fa2\PCT$E@A2.JPG,"[C, P, T]",E,A2,True,False,False,False,False,False,True,False,True,False,3.426981
POMCT$E@A2.JPG,ImageBase\Ext\Fa2\POMCT$E@A2.JPG,"[C, M, O, P, T]",E,A2,True,False,False,True,False,True,True,False,True,False,3.179371
PTO$E@A2.JPG,ImageBase\Ext\Fa2\PTO$E@A2.JPG,"[O, P, T]",E,A2,False,False,False,False,False,True,True,False,True,False,3.345244
TCM$E@A2.JPG,ImageBase\Ext\Fa2\TCM$E@A2.JPG,"[C, M, T]",E,A2,True,False,False,True,False,False,False,False,True,False,3.427424
TCMP$E@A2.JPG,ImageBase\Ext\Fa2\TCMP$E@A2.JPG,"[C, M, P, T]",E,A2,True,False,False,True,False,False,True,False,True,False,3.245831


# Metadados 

In [4]:
x.meta

{'Classes': ['C', 'F', 'H', 'M', 'N', 'O', 'P', 'R', 'T', 'X'],
 'Número de Classes': 10,
 'Fundos': ['Listrado',
  'Branco',
  'Madeira',
  'Marrom',
  'Estampado',
  'Azul Claro',
  'Grama',
  'Azul'],
 'Iluminações': ['Interna', 'Externa'],
 'Número de Imagens': 213,
 'Tamanho Total (MB)': 486.27}

# Metadados das Classes

In [5]:
x.info_classes

,Classe,Código,Número de Amostras
C,Carregador,0,47
F,Faca,1,88
H,Chinelo,2,69
M,Mouse,3,46
N,Controle,4,88
O,Cortador de Unha,5,55
P,Protetor Solar,6,51
R,Parafuso,7,88
T,Tesoura,8,51
X,Chave,9,100


# Metadados das Iluminações

In [6]:
x.info_iluminacoes

,Iluminação,Número de Amostras
I,Interna,119
E,Externa,94


# Metadados dos Fundos

In [7]:
x.info_fundos

,Fundo,Número de Amostras
L,Listrado,49
B,Branco,44
M,Madeira,36
F,Marrom,28
R,Estampado,18
A2,Azul Claro,14
G,Grama,13
A,Azul,11


# RGB2gray

In [8]:
def RGB2gray(img):
    return cvtColor(img, COLOR_BGR2GRAY)

# Contrast Stretching

In [9]:
def contrast_stretching(img):
    intensidade_minima = img.min()
    intensidade_maxima = img.max()
    return (img - intensidade_minima) / (intensidade_maxima - intensidade_minima) * 255

# Log

In [10]:
def log(img, base):
    return np.log(img) / np.log(base)

# Exponencial

In [11]:
def exp(img, c):
    return np.power(img, c)

# Laplaciano

In [12]:
def convolution2d(image, kernel):
    m, n = kernel.shape
    if (m == n):
        y, x = image.shape
        y = y - m + 1
        x = x - m + 1
        new_image = np.zeros((y,x))
        for i in range(y):
            for j in range(x):
                new_image[i][j] = np.sum(image[i:i+m, j:j+m]*kernel)
    return new_image

In [13]:
def laplaciano(img):
    return convolution2d(img, np.array([[1,1,1], [1,-8,1], [1,1,1]]))

# Filtro Média

In [14]:
def filtro_media(img):
    return convolution2d(img, 1/49 * np.ones((7,7)))

# Processando o DataSet para Todas as Funções

In [15]:
funcs_name = ['contrast_stretching', 'log', 'exp', 'laplaciano', 'filtro_media']
cs = contrast_stretching
def process():
    c = 1
    for name in x.data.index:
        img = x.getById(name)
        img = cvtColor(img, COLOR_BGR2GRAY)
        img = resize(img, (0,0), fx=.35, fy=.35)
        
        imwrite('./augmentedDataSet/contrast_stretching/' + name, contrast_stretching(img))
        imwrite('./augmentedDataSet/log/' + name, (log(img, 2)))
        imwrite('./augmentedDataSet/exp/' + name, (exp(img, 1.5)))
        imwrite('./augmentedDataSet/laplaciano/' + name, laplaciano(img))
        imwrite('./augmentedDataSet/media/' + name, filtro_media(img))
        
        if c % 10 == 0: print(c)
        c+=1

In [ ]:
process()

10
20
30
40
50
60
70
80
90


C:\Users\WILLIA~1.GAR\AppData\Local\Temp/ipykernel_8496/3323199778.py:2: RuntimeWarning: divide by zero encountered in log
  return np.log(img) / np.log(base)


100
110
120
130
140
150
160
170
180
